# Neural Ordinary Differential Equations

На конференции NIPS 2019 года была представлена статья,
которая называется 'Нейронные обычные дифференциальные уравнения'.
Эта статья была в числе 4 из 4854 статей, которая получила награду
'Best paper'. В своей работе я постараюсь разобрать метод, предлагаемый
авторами и сравнить данную архитектуру с уже существующими решениями

### Вступление

Что происходит в обычных нейронных сетях? В них есть слои(скрытые состояния) $ h_t $ и переходы между этими состояниями
 происходят с помощью умножений на соответствующие матрицы весов $ W_t $.
<img src=assets/mlp.png width=600 ></img>

У остаточных нейронных сетей принцип действия такой же, за исключением того, что на выходе из слоя добавляется
еще и исходное состояние: <img src=assets/res_learn.png width=600 ></img>

Идея этого заключается в том, что мы не очень хотим чтобы наше состояние кардинально отличалось от
предыдущего на каждом шаге.

Состояния остаточных нейронных сетей описываются следующей формулой: $h_{t+1} = h_{t} + f(h_{t}, \theta_{t})$, где $t \in \{0...T\}$ -
это номер блока, $ \theta_t $ - матрица весов, а $f $ - функция активации соответствующего слоя.

А что если $ T $ устремить к бесконечности, а $ t_{i} - t_{i-1} $ , то есть, шаг, устремить к нулю?
Тогда это можно рассматривать как некий временной процесс.
Следовательно, это можно записать в виде дифференциального уравнения: $\frac{dh(t)}{dt} = f(h(t), t, \theta)$

Тогда, решая это дифференциальное уравнение можно восстановить искомую зависимость.


Как решать такое дифференциальное уравнение?
В исходной статье используется метод Эйлера. Как он работает: <img src=assets/euler.jpg width=600 ></img>
Численное решение задается формулой $ y_i = y_{i -1} + hf(x_{i-1}, y_{i -1}) $

То есть, на каждой итерации мы делаем маленький шаг в сторону градиента функции. В конечном итоге
после всех итераций мы восстановим искомую функцию.

Обучение такой нейронной сети заключается в том, что нам необходимо найти такие веса $ \theta $ , чтобы
получить правильные градиенты $\frac{dh(t)}{dt} $ такие, что когда мы решаем ОДУ, мы получим правильную функцию
, которая в конечный момент времени $ t $ , то есть, на выходном
слое выдаст правильный ответ с желаемой точностью.

#### Как мы будем обучать такую нейросеть?

Наша нейросеть определяет переходы от одного состояния к следующему. У нее есть параметры $ \theta $ , которые
нам и нужно обучить. А обучать мы их будем с помощью обычного градиентного спуска, прямо как в
классической нейронной сети. Но не все так просто.

<img src=assets/loss.png width=600 ></img>

Функция потери в свою очередь задается следующим образом:
$ L(z(t_1)) = L(z(t_0) + \int_{t_0}^{t_1} f(z(t), t, \theta) dt) = L(ODESolve(z(t_0), f, t_0, t_1, \theta))  $

Таким образом, чтобы оптимизировать $ L $ нам нужны произодные $ \frac{dL}{d\theta} $ .
Чтобы это сделать нужно определить как функция потерь зависит от $ z(t) $ в каждый момент.
Эта зависимость обозначается как $ a(t) = \frac{dL}{dz(t)} $ и называется *сопряженным* состоянием.

Тогда получим: $ \frac{d a(t)}{d t} = -a(t) \frac{\partial f(z(t), t, \theta)}{\partial z}
 $ (вывод этой формулы можно посмотреть в оригинальной статье).

Решая этот дифур можно получить все необходимые частные производные:

$
\frac{\partial L}{\partial z(t_0)} = \int_{t_1}^{t_0} a(t) \frac{\partial f(z(t), t, \theta)}{\partial z} dt
$

$
\frac{\partial L}{\partial \theta} = \int_{t_1}^{t_0} a(t) \frac{\partial f(z(t), t, \theta)}{\partial \theta} dt
$

$
\frac{\partial L}{\partial t_0} = \int_{t_1}^{t_0} a(t) \frac{\partial f(z(t), t, \theta)}{\partial t} dt
$

Все эти частные производные могут быть посчитаны за один вызов ODESolve:
<img src=assets/algo.png width=600 ></img>

### Практика

Авторы статьи говорят, что такая архитектура требует константной памяти, адаптирует стратегию вычисления
к каждым входным данным и позволяет выгодно обменивать численную точность на скорость

Это звучит очень круто, но так ли оно на самом деле?

Посмотрим как оно умеет решать диффуры.

In [ ]:
!git clone https://github.com/rtqichen/torchdiffeq.git

In [ ]:
!cd torchdiffeq && pip install -e .

In [ ]:
import os
import time
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
# from torchdiffeq.torchdiffeq import odeint
from torchdiffeq import odeint
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Зададим систему дифференциальных уравнений следующего вида:
$Y' = AY$ где $A = \begin{bmatrix}
a & b \\
c & d \end{bmatrix}$ и $Y$ = $\begin{bmatrix} y_1 (t) \\ y_2 (t)
\end{bmatrix}$

Пусть $A = \begin{bmatrix}
0 & -2 \\
2 & 0
\end{bmatrix}$
и $Y(0) = \begin{bmatrix}
2 \\
0
\end{bmatrix}$
Тогда решая систему получим
$ Y = \begin{bmatrix}
2\cos(t) \\
2\sin(t)
\end{bmatrix} $, оно же является параметрическим уравнением эллипса

In [ ]:
true_y0 = torch.tensor([[2., 0.]]).to(device)
t = torch.linspace(0., 25., 1000).to(device)
true_A = torch.tensor([[0, -2.], [2., 0]]).to(device)

class Lambda(nn.Module):

    def forward(self, t, y):
        torch.mm(y, true_A)

with torch.no_grad:
    true_y = odeint(Lambda(), true_y0, t, method="dopr15")

На самом деле, метод Эйлера решения дифференциальных уравнений сейчас считается
неточным и уже устаревшим.
Поэтому, внутри функции *odeint* реализованы более
сложные методы решения. Сейчас выведем на плоскость решение этого диффура.
Напомню, что должен получиться эллипс.

In [ ]:
plt.figure()
plt.plot(t.numpy(), true_y.numpy()[:, 0, 0], t.numpy(), true_y.numpy()[:, 0, 1], 'g-')
plt.show()

## Источники

Исходная статья - https://arxiv.org/abs/1806.07366

Авторский репозиторий - https://github.com/rtqichen/torchdiffeq

Имплементация ODENet в tensorflow - https://github.com/jason71995/Keras_ODENet

Имплементация ResNet в tensorflow - https://github.com/SeHwanJoo/cifar10-ResNet-tensorflow
